In [1]:
!pip install datasets evaluate accelerate
!pip install sacrebleu jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   

In [2]:
import os

import torch
from datasets import load_dataset
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
import evaluate
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
output_dir = '/content/saved_model_weights'
os.makedirs(output_dir, exist_ok=True)

In [5]:
MODEL_NAME = "vinai/bartpho-syllable"
MAX_LENGTH = 512

In [6]:
def preprocess_function(examples):
    # Tokenize the text and apply truncation
    return tokenizer(examples["error"],
                     text_target=examples["original"],
                     max_length=MAX_LENGTH,
                     truncation=True,
                     )

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"sacrebleu": result["score"]}

In [7]:
# Define paths to your CSV files
train_path = '/content/drive/MyDrive/Colab Notebooks/BARTpho/data_preparation/train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/BARTpho/data_preparation/test.csv'

dataset = load_dataset("csv", data_files={"train": train_path, "test": test_path})

print(dataset['train'])

print(dataset['test'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['original', 'error'],
    num_rows: 16099
})
Dataset({
    features: ['original', 'error'],
    num_rows: 4025
})


In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

input = dataset["train"][-1]['error']
output = dataset["train"][-1]['original']

# Tokenizing Input Data
inputs = tokenizer(
    input,
    text_target=output,
    max_length=MAX_LENGTH,
    truncation=True
)

inputs, len(inputs['input_ids'])

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

({'input_ids': [0, 1172, 1886, 427, 434, 3270, 309, 155, 4, 10, 71, 478, 731, 7, 197, 924, 706, 197, 331, 63, 1037, 52, 146, 739, 753, 989, 124, 4, 90, 349, 269, 672, 262, 141, 4, 344, 15, 33200, 52, 3682, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 1172, 349, 427, 326, 309, 155, 4, 10, 71, 681, 7, 197, 924, 706, 197, 331, 63, 1037, 52, 146, 1278, 124, 4, 90, 349, 269, 672, 262, 141, 4, 344, 15, 333, 52, 3682, 5, 2]},
 42)

In [9]:
# Apply tokenization in a batched manner for efficiency
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/16099 [00:00<?, ? examples/s]

Map:   0%|          | 0/4025 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

metric = evaluate.load("sacrebleu")

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [12]:
args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir="./checkpoints",
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=10,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=1500,
    logging_dir="./logs",
    logging_steps=1500,
    predict_with_generate=True,
    save_strategy="no",
    resume_from_checkpoint=False
)

In [13]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Sacrebleu
1500,1.094100,0.120982,34.141534


In [ ]:
model.save_pretrained(output_dir)

In [ ]:
corrector = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

MAX_LENGTH = 512

# Define the text samples
texts = [
    "Nỗi buồn là giớt sương đg rơi trên mắt e vo anh",
    "Hàn mi cong lun linh trng anh dương vừa tan",
    "Nhìn em như  dang khóc ò ",
    "Khi bắt đầuuu, cả hai ta đi khôi đau dơn lúc gặp nhau",
    "ơi những truyến đi, thế dang bộn bề và lo như gặp nhau",
]

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred['generated_text'])

In [ ]:
!cp -r "/content/saved_model_weights"  "/content/drive/MyDrive/Colab Notebooks/BARTpho/saved_model_weights"